In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from moviepy.editor import VideoFileClip

# Step 1: Data Preparation and Frame Extraction

def extract_frames(video_path, output_frame_folder):
    # Create output folder if not exists
    if not os.path.exists(output_frame_folder):
        os.makedirs(output_frame_folder)
    
    # Open video file
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    # Read until video is completed
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Save frame as image
        frame_name = os.path.join(output_frame_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_name, frame)
        frame_count += 1
    
    # Release the video capture object
    cap.release()

# Function to extract audio
def extract_audio(video_path, output_audio_folder):
    # Create output folder if not exists
    if not os.path.exists(output_audio_folder):
        os.makedirs(output_audio_folder)
    
    # Open video file
    clip = VideoFileClip(video_path)
    
    # Check if the video has audio
    if clip.audio is not None:
        audio_path = os.path.join(output_audio_folder, "audio.mp3")
        
        # Write audio file
        clip.audio.write_audiofile(audio_path, fps=44100)
    
    # Close the video clip
    clip.close()

# Provide paths to violence and non-violence video folders
violence_folder = "D:/CSE449B/Real Life Violence Dataset/violence"
non_violence_folder = "D:/CSE449B/Real Life Violence Dataset/non_violence"

# Extract frames and audio from violence videos
for video_file in os.listdir(violence_folder):
    video_path = os.path.join(violence_folder, video_file)
    output_frame_folder = os.path.join("violence_frames", os.path.splitext(video_file)[0])
    output_audio_folder = os.path.join("violence_audio", os.path.splitext(video_file)[0])
    extract_frames(video_path, output_frame_folder)
    extract_audio(video_path, output_audio_folder)

# Extract frames and audio from non-violence videos
for video_file in os.listdir(non_violence_folder):
    video_path = os.path.join(non_violence_folder, video_file)
    output_frame_folder = os.path.join("non_violence_frames", os.path.splitext(video_file)[0])
    output_audio_folder = os.path.join("non_violence_audio", os.path.splitext(video_file)[0])
    extract_frames(video_path, output_frame_folder)
    extract_audio(video_path, output_audio_folder)

In [ ]:
import os
import cv2
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from moviepy.editor import VideoFileClip
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet_input
from tqdm import tqdm

# Load pre-trained ResNet50 model without top classification layer
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Function to extract visual features from video frames using ResNet50
def extract_visual_features(frame_folder):
    visual_features = []
    for frame_file in tqdm(os.listdir(frame_folder), desc="Extracting visual features"):
        frame_path = os.path.join(frame_folder, frame_file)
        frame = cv2.imread(frame_path)
        frame = cv2.resize(frame, (96, 96))  # Resize frame to 96x96
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frame = preprocess_resnet_input(frame)  # Preprocess frame for ResNet50
        features = resnet_model.predict(np.expand_dims(frame, axis=0))  # Extract features
        visual_features.append(features.flatten())
    return np.array(visual_features)

# Function to extract audio features using librosa directly from video files
def extract_audio_features(video_path, sr=22050, n_mfcc=2048):
    try:
        video = VideoFileClip(video_path)
        if video.audio is None:
            print(f"No audio found in {video_path}")
            return None
        audio_array = video.audio.to_soundarray()  # Extract audio as numpy array
        audio_features = librosa.feature.mfcc(y=audio_array[:, 0], sr=sr, n_mfcc=n_mfcc)  # Extract MFCC features
        return np.mean(audio_features, axis=1)  # Take mean of MFCC features
    except Exception as e:
        print(f"Error extracting audio features from {video_path}: {e}")
        return None

# Provide paths to violence and non-violence video folders
violence_folder = "D:/CSE449B(2)/Real Life Violence Dataset/violence"
non_violence_folder = "D:/CSE449B(2)/Real Life Violence Dataset/non_violence"

# Extract features from violence videos
violence_visual_features = []
violence_audio_features = []

for video_file in tqdm(os.listdir(violence_folder), desc="Processing violence videos"):
    video_path = os.path.join(violence_folder, video_file)
    output_frame_folder = os.path.join("violence_frames", os.path.splitext(video_file)[0])

    # Extract visual features
    visual_features = extract_visual_features(output_frame_folder)
    violence_visual_features.extend(visual_features)

    # Extract audio features
    audio_features = extract_audio_features(video_path)
    if audio_features is not None:
        violence_audio_features.append(audio_features)

# Extract features from non-violence videos
non_violence_visual_features = []
non_violence_audio_features = []

for video_file in tqdm(os.listdir(non_violence_folder), desc="Processing non-violence videos"):
    video_path = os.path.join(non_violence_folder, video_file)
    output_frame_folder = os.path.join("non_violence_frames", os.path.splitext(video_file)[0])

    # Extract visual features
    visual_features = extract_visual_features(output_frame_folder)
    non_violence_visual_features.extend(visual_features)

    # Extract audio features
    audio_features = extract_audio_features(video_path)
    if audio_features is not None:
        non_violence_audio_features.append(audio_features)

# Convert lists to numpy arrays
violence_visual_features = np.array(violence_visual_features)
violence_audio_features = np.array(violence_audio_features)
non_violence_visual_features = np.array(non_violence_visual_features)
non_violence_audio_features = np.array(non_violence_audio_features)

# Now you have extracted visual and audio features for violence and non-violence videos.

In [21]:
print(np.shape(violence_visual_features))
print(np.shape(violence_audio_features))
print(np.shape(non_violence_visual_features))
print(np.shape(non_violence_audio_features))

(4098, 2048)
(22, 128)
(3809, 2048)
(28, 128)


In [ ]:
# import os
# import cv2
# import numpy as np
# import librosa
# from sklearn.model_selection import train_test_split
# from moviepy.editor import VideoFileClip
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet_input
# from tqdm import tqdm

# # Load pre-trained ResNet50 model without top classification layer
# resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# # Function to preprocess visual features (resize to a consistent dimension)
# def preprocess_visual_features(features, target_shape=(96, 96)):
#     preprocessed_features = []
#     for feature in features:
#         resized_feature = cv2.resize(feature, target_shape)
#         preprocessed_features.append(resized_feature)
#     return np.array(preprocessed_features)

# # Function to preprocess audio features (pad or truncate to a consistent length)
# def preprocess_audio_features(features, target_length=2048):
#     preprocessed_features = []
#     for feature in features:
#         # If feature length is greater than target length, truncate; if less, pad with zeros
#         if len(feature) > target_length:
#             preprocessed_feature = feature[:target_length]
#         else:
#             preprocessed_feature = np.pad(feature, (0, target_length - len(feature)), mode='constant')
#         preprocessed_features.append(preprocessed_feature)
#     return np.array(preprocessed_features)

# # Function to extract visual features from video frames using ResNet50
# def extract_visual_features(frame_folder):
#     visual_features = []
#     for frame_file in tqdm(os.listdir(frame_folder), desc="Extracting visual features"):
#         frame_path = os.path.join(frame_folder, frame_file)
#         frame = cv2.imread(frame_path)
#         frame = cv2.resize(frame, (96, 96))  # Resize frame to 96x96
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
#         frame = preprocess_resnet_input(frame)  # Preprocess frame for ResNet50
#         features = resnet_model.predict(np.expand_dims(frame, axis=0))  # Extract features
#         visual_features.append(features.flatten())
#     return np.array(visual_features)

# # Function to extract audio features using librosa directly from video files
# def extract_audio_features(video_path, sr=22050, n_mfcc=20):
#     try:
#         video = VideoFileClip(video_path)
#         if video.audio is None:
#             print(f"No audio found in {video_path}")
#             return None
#         audio_array = video.audio.to_soundarray()  # Extract audio as numpy array
#         audio_features = librosa.feature.mfcc(y=audio_array[:, 0], sr=sr, n_mfcc=n_mfcc)  # Extract MFCC features
#         return np.mean(audio_features, axis=1)  # Take mean of MFCC features
#     except Exception as e:
#         print(f"Error extracting audio features from {video_path}: {e}")
#         return None

# # Provide paths to violence and non-violence video folders
# violence_folder = "D:/CSE449B(2)/Real Life Violence Dataset/violence"
# non_violence_folder = "D:/CSE449B(2)/Real Life Violence Dataset/non_violence"

# # Extract features from violence videos
# violence_visual_features = []
# violence_audio_features = []

# for video_file in tqdm(os.listdir(violence_folder), desc="Processing violence videos"):
#     video_path = os.path.join(violence_folder, video_file)
#     output_frame_folder = os.path.join("violence_frames", os.path.splitext(video_file)[0])

#     # Extract visual features
#     visual_features = extract_visual_features(output_frame_folder)
#     violence_visual_features.extend(visual_features)

#     # Extract audio features
#     audio_features = extract_audio_features(video_path)
#     if audio_features is not None:
#         violence_audio_features.append(audio_features)

# # Extract features from non-violence videos
# non_violence_visual_features = []
# non_violence_audio_features = []

# for video_file in tqdm(os.listdir(non_violence_folder), desc="Processing non-violence videos"):
#     video_path = os.path.join(non_violence_folder, video_file)
#     output_frame_folder = os.path.join("non_violence_frames", os.path.splitext(video_file)[0])

#     # Extract visual features
#     visual_features = extract_visual_features(output_frame_folder)
#     non_violence_visual_features.extend(visual_features)

#     # Extract audio features
#     audio_features = extract_audio_features(video_path)
#     if audio_features is not None:
#         non_violence_audio_features.append(audio_features)

# # Convert lists to numpy arrays
# violence_visual_features = np.array(violence_visual_features)
# violence_audio_features = np.array(violence_audio_features)
# non_violence_visual_features = np.array(non_violence_visual_features)
# non_violence_audio_features = np.array(non_violence_audio_features)

# # Preprocess visual features
# violence_visual_features = preprocess_visual_features(violence_visual_features)
# non_violence_visual_features = preprocess_visual_features(non_violence_visual_features)

# # Preprocess audio features
# violence_audio_features = preprocess_audio_features(violence_audio_features)
# non_violence_audio_features = preprocess_audio_features(non_violence_audio_features)

# # Now both visual and audio features should have consistent dimensions

In [25]:
print(np.shape(violence_visual_features))
print(np.shape(non_violence_visual_features))
print(np.shape(violence_audio_features))
print(np.shape(non_violence_audio_features))

(4098, 96, 96)
(3809, 96, 96)
(22, 2048)
(28, 2048)


In [5]:
import os
import cv2
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from moviepy.editor import VideoFileClip
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet_input
from tqdm import tqdm

# Load pre-trained ResNet50 model without top classification layer
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Function to extract visual features from video frames using ResNet50
def extract_visual_features(frame_folder):
    visual_features = []
    for frame_file in tqdm(os.listdir(frame_folder), desc="Extracting visual features"):
        frame_path = os.path.join(frame_folder, frame_file)
        frame = cv2.imread(frame_path)
        frame = cv2.resize(frame, (96, 96))  # Resize frame to 96x96
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frame = preprocess_resnet_input(frame)  # Preprocess frame for ResNet50
        features = resnet_model.predict(np.expand_dims(frame, axis=0))  # Extract features
        visual_features.append(features.flatten())
    return np.array(visual_features)
# def extract_visual_features(video_path):
#     visual_features = []
#     cap = cv2.VideoCapture(video_path)
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         # Resize frame to desired dimensions
#         frame = cv2.resize(frame, (96, 96))
#         # Convert BGR to RGB
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         # Preprocess frame for ResNet50
#         frame = preprocess_resnet_input(frame)
#         # Extract features using ResNet50
#         features = resnet_model.predict(np.expand_dims(frame, axis=0))
#         visual_features.append(features.flatten())
#     cap.release()
#     return np.array(visual_features)


# Function to extract audio features using librosa directly from video files
def extract_audio_features(video_path, sr=22050, n_mfcc=128):
    try:
        video = VideoFileClip(video_path)
        if video.audio is None:
            print(f"No audio found in {video_path}")
            return None
        audio_array = video.audio.to_soundarray()[:, 0]  # Extract audio as numpy array (mono channel)
        audio_features = librosa.feature.mfcc(y=audio_array, sr=sr, n_mfcc=n_mfcc)  # Extract MFCC features
        return audio_features.T  # Transpose to match the shape of visual features
    except Exception as e:
        print(f"Error extracting audio features from {video_path}: {e}")
        return None

# Ensure that the number of visual features matches the number of audio features for each video
# Check if there are missing features for any video and remove those videos from the dataset
def remove_missing_features(visual_features, audio_features):
    if len(visual_features) != len(audio_features):
        print("Number of visual and audio features do not match for some videos. Removing incomplete data...")
        min_length = min(len(visual_features), len(audio_features))
        visual_features = visual_features[:min_length]
        audio_features = audio_features[:min_length]
    return visual_features, audio_features

# Provide paths to violence and non-violence video folders
violence_folder = "D:/CSE449B(2)/Real Life Violence Dataset/violence"
non_violence_folder = "D:/CSE449B(2)/Real Life Violence Dataset/non_violence"

# Extract features from violence videos
violence_visual_features = []
violence_audio_features = []

for video_file in tqdm(os.listdir(violence_folder), desc="Processing violence videos"):
    video_path = os.path.join(violence_folder, video_file)
    output_frame_folder = os.path.join("violence_frames", os.path.splitext(video_file)[0])

    # Extract visual features
    visual_features = extract_visual_features(output_frame_folder)
    
    # Extract audio features
    audio_features = extract_audio_features(video_path)
    
    if visual_features.shape[0] != 0 and audio_features is not None:
        visual_features, audio_features = remove_missing_features(visual_features, audio_features)
        violence_visual_features.extend(visual_features)
        violence_audio_features.extend(audio_features)

# Extract features from non-violence videos
non_violence_visual_features = []
non_violence_audio_features = []

for video_file in tqdm(os.listdir(non_violence_folder), desc="Processing non-violence videos"):
    video_path = os.path.join(non_violence_folder, video_file)
    output_frame_folder = os.path.join("non_violence_frames", os.path.splitext(video_file)[0])

    # Extract visual features
    visual_features = extract_visual_features(output_frame_folder)
    
    # Extract audio features
    audio_features = extract_audio_features(video_path)
    
    if visual_features.shape[0] != 0 and audio_features is not None:
        visual_features, audio_features = remove_missing_features(visual_features, audio_features)
        non_violence_visual_features.extend(visual_features)
        non_violence_audio_features.extend(audio_features)

# Convert lists to numpy arrays
violence_visual_features = np.array(violence_visual_features)
violence_audio_features = np.array(violence_audio_features)
non_violence_visual_features = np.array(non_violence_visual_features)
non_violence_audio_features = np.array(non_violence_audio_features)

# Check the shapes of the extracted features
print("Violence visual features shape:", violence_visual_features.shape)
print("Violence audio features shape:", violence_audio_features.shape)
print("Non-violence visual features shape:", non_violence_visual_features.shape)
print("Non-violence audio features shape:", non_violence_audio_features.shape)

Processing violence videos:   0%|          | 0/30 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


Processing non-violence videos: 100%|██████████| 30/30 [08:51<00:00, 17.72s/it]

Number of visual and audio features do not match for some videos. Removing incomplete data...


Violence visual features shape: (3083, 2048)
Violence audio features shape: (3083, 128)
Non-violence visual features shape: (3534, 2048)
Non-violence audio features shape: (3534, 128)


In [6]:
print("Violence visual features shape:", violence_visual_features.shape)
print("Violence audio features shape:", violence_audio_features.shape)
print("Non-violence visual features shape:", non_violence_visual_features.shape)
print("Non-violence audio features shape:", non_violence_audio_features.shape)

Violence visual features shape: (3083, 2048)
Violence audio features shape: (3083, 128)
Non-violence visual features shape: (3534, 2048)
Non-violence audio features shape: (3534, 128)


In [7]:
print("Shape of one audio sample:", violence_audio_features[0].shape)


Shape of one audio sample: (128,)
